# After refactoring...

In [ ]:
import signalflow as sf
from pixasonics.core import App, Mapper
from pixasonics.features import MeanPixelValue
from pixasonics.synths import Theremin

img_path = "images/cellular_dataset/Timepoint_001_220518-ST_C03_s1.jpg"

# Create app
app = App()
app.load_image(img_path) # Load image

# Create objects
mean_pix = MeanPixelValue()
theremin = Theremin()
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_high=1000)

# Add objects to app
app.attach_feature(mean_pix)
app.attach_synth(theremin)
app.attach_mapper(pix2freq)


In [ ]:
pix2freq.ui

In [ ]:
# test adding another mapping: mean pixel value to amplitude with an exponential curve
pix2amp = Mapper(mean_pix, theremin["amplitude"], exponent=2)
app.attach_mapper(pix2amp)
pix2amp.ui

In [4]:
pix2pan = Mapper(mean_pix, theremin["panning"])
app.attach_mapper(pix2pan)

In [ ]:
app.mappers

In [ ]:
# remove a mapper
app.detach_mapper(pix2amp)
app.mappers

In [ ]:
# remove old frequency mapping and add a new one with different scaling
app.detach_mapper(pix2freq)
pix2freq = Mapper(mean_pix, theremin["frequency"], exponent=2, out_low=100, out_high=2000)
app.attach_mapper(pix2freq)
app.mappers

In [ ]:
graph = sf.AudioGraph.get_shared_graph()
print(graph.structure)
print(graph.status)

# Import widget stuff

In [1]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion

# Docs Examples

In [10]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))

In [ ]:
grid = GridspecLayout(4, 3, height='300px')
grid[:3, 1:] = create_expanded_button('One', 'success')
grid[:, 0] = create_expanded_button('Two', 'info')
grid[3, 1] = create_expanded_button('Three', 'warning')
grid[3, 2] = create_expanded_button('Four', 'danger')
grid

In [ ]:
item_layout = Layout(height='100px', min_width='40px')
items = [Button(layout=item_layout, description=str(i), button_style='warning') for i in range(40)]
box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='500px',
                    height='',
                    flex_flow='row',
                    display='flex')
carousel = Box(children=items, layout=box_layout)
VBox([Label('Scroll horizontally:'), carousel])

In [ ]:
header  = Button(description='Header',
                 layout=Layout(width='auto', grid_area='header'),
                 style=ButtonStyle(button_color='lightblue'))
main    = Button(description='Main',
                 layout=Layout(width='auto', grid_area='main'),
                 style=ButtonStyle(button_color='moccasin'))
sidebar = Button(description='Sidebar',
                 layout=Layout(width='auto', grid_area='sidebar'),
                 style=ButtonStyle(button_color='salmon'))
footer  = Button(description='Footer',
                 layout=Layout(width='auto', grid_area='footer'),
                 style=ButtonStyle(button_color='olive'))

GridBox(children=[header, main, sidebar, footer],
        layout=Layout(
            width='50%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "header header header header"
            "main main . sidebar "
            "footer footer footer footer"
            ''')
       )

In [ ]:
GridBox(children=[Button(layout=Layout(width='auto', height='auto'),
                         style=ButtonStyle(button_color='darkseagreen')) for i in range(9)
                 ],
        layout=Layout(
            width='50%',
            grid_template_columns='100px 50px 100px',
            grid_template_rows='80px auto 80px', 
            grid_gap='5px 10px')
       )

# Mapper card proto

In [ ]:
mapper_label = Label(value="Mapper", style=dict(
    font_weight='bold',
    font_size='2em'))
mapper_id = Label(value="# ID", style=dict(
    font_weight='bold',
    font_size='1em',
    text_color='gray'))

top_row = Box([mapper_label, mapper_id], layout=Layout(justify_content='space-between'))

from_label = Label(value="From:")
from_value = Label(value="MeanPixelValue")
from_row = Box([from_label, from_value], layout=Layout(justify_content='space-between'))

to_label = Label(value="To:")
to_value = Label(value="Theremin Frequency")
to_row = Box([to_label, to_value], layout=Layout(justify_content='space-between'))

detach_btn = Button(description="Detach", button_style='danger', icon='unlink')
detach_btn.on_click(lambda x: print("Detaching mapper"))
detach_row = Box([detach_btn], layout=Layout(justify_content='flex-end'))

mapper_card = GridBox(children=[top_row, from_row, to_row, detach_row],
                        layout=Layout(
                            width='auto', 
                            grid_template_columns='auto', 
                            grid_template_rows='2fr 1.5fr 1.5fr 1fr',
                            max_width='300px',
                            min_height='200px',
                            border='2px solid black',
                            padding='5px',
                            margin='5px'))

mapper_card


# Carousel proto

In [ ]:
# Create a vertical carousel of mapper cards
mapper_cards = [mapper_card for i in range(10)]
mapper_carousel = Box(children=mapper_cards, layout=Layout(overflow='scroll',
                                                            border='3px solid black',
                                                            padding='5px',
                                                            width='auto',
                                                            max_width='300px',
                                                            max_height='500px',
                                                            flex_flow='column',
                                                            display='flex'))
mapper_carousel

In [ ]:
# access the children of the carousel
mapper_carousel.children

# remove a card
mapper_carousel.children = mapper_carousel.children[:-1]

mapper_carousel

# Probe settings ui proto

In [ ]:
probe_w_label = Label(value="Probe Width:")
probe_w_slider = IntSlider(value=50, min=1, max=500, step=1)
probe_w_box = Box(
    [probe_w_label, probe_w_slider], 
    layout=Layout(
        justify_content='space-around', 
        align_items='flex-start', 
        flex_flow='column',
        padding='5px'))

probe_h_label = Label(value="Probe Height:")
probe_h_slider = IntSlider(value=50, min=1, max=500, step=1)
probe_h_box = Box(
    [probe_h_label, probe_h_slider], 
    layout=Layout(
        justify_content='space-around', 
        align_items='flex-start', 
        flex_flow='column',
        padding='5px'))

probe_box = Box(
    [probe_w_box, probe_h_box], 
    layout=Layout(
        justify_content='space-around', 
        align_items='flex-start', 
        flex_flow='column',
        border='2px solid black',
        padding='5px',
        margin='5px'))

probe_box

# Audio settings proto

In [ ]:
audio_switch = ToggleButton(
    value=False,
    description='Audio',
    tooltip='Enable/disable audio processing',
    icon='volume-up',
    layout=Layout(
        width='auto', 
        max_width='100px',
        height='auto')
)

master_volume_label = Label(value="Master Volume (dB):")

master_volume_slider = FloatSlider(
    value=0,
    min=-36,
    max=0,
    step=0.01,
    orientation='horizontal',
    layout=Layout(width='100%', height='auto')
)

master_volume_box = Box(
    [master_volume_label, master_volume_slider],
    layout=Layout(
        justify_content='space-around', 
        align_items='flex-start', 
        flex_flow='column',
        padding='5px'))

audio_controls = GridBox(
    children=[audio_switch, master_volume_box],
    layout=Layout(
        width='auto', 
        grid_template_columns='auto auto', 
        grid_template_rows='auto',
        border='2px solid black',
        padding='5px',
        margin='5px'))

audio_controls


# App layout proto

In [ ]:
app_canvas_dummy = Button(description="App Canvas", layout=Layout(width='500px', height='500px'))

app_settings = Accordion(
    children=[probe_box, audio_controls, mapper_carousel, mapper_carousel, mapper_carousel],
    titles=('Probe', 'Audio', "Features", "Synths", "Mappers"),
    layout=Layout(width='400px', min_width='300px', max_width='400px'))

app_settings_container = Box(
    [app_settings], 
    layout=Layout(
        overflow='scroll',
        border='3px solid black',
        padding='5px',
        max_height='500px',))

app_ui = Box(
    [app_canvas_dummy, app_settings_container], 
    layout=Layout(
        width='auto', 
        height='auto', 
        justify_content='space-around'))

app_ui

In [1]:
from ipywidgets import ToggleButton, Button, ButtonStyle, Layout, jslink, IntText, IntSlider, FloatSlider, GridBox, GridspecLayout, Box, VBox, Label, Accordion
from pixasonics.ui import AppUI, ProbeSettings, AudioSettings

probe_settings = ProbeSettings()
audio_settings = AudioSettings()

app_ui = AppUI(
    probe_settings,
    audio_settings
    )

app_ui()

Box(children=(Box(layout=Layout(height='500px', width='500px')), Box(children=(Accordion(children=(Box(childre…

In [2]:
# add a new canvas
test_canvas = Button(description="Test Canvas", layout=Layout(width='500px', height='500px'))
app_ui.box.children[0].children = [test_canvas]

In [5]:
# add buttons to the features carousel
features_carousel = app_ui.box.children[1].children[0].children[2]
features_carousel.children = list(features_carousel.children) + [Button(description=f"Feature {i}") for i in range(10)]

In [4]:
# delete all buttons from the features carousel
features_carousel.children = []

In [6]:
# function to delete the nth element from the features carousel
def delete_nth(n):
    features_carousel.children = features_carousel.children[:n] + features_carousel.children[n+1:]

In [7]:
delete_nth(3)

In [8]:
# function to delete an element with a matching description
def delete_by_description(description):
    for i, child in enumerate(features_carousel.children):
        if child.description == description:
            delete_nth(i)
            break

In [9]:
delete_by_description("Feature 5")

In [10]:
# override the probe w slider callback
# WRONG WAY
def probe_w_slider_callback(change):
    print(f"Probe Width: {change.new}")

app_ui.probe_settings.w_slider_callback = probe_w_slider_callback

In [ ]:
# CORRECT WAY
def probe_w_slider_callback(change):
    print(f"Probe Width: {change.new}")

probe_w_slider = app_ui.children

In [11]:
def find_widget_by_description_recursive(container, widget_type, description):
    """
    Recursively search through a container for a widget of a specific type with a specific description.

    Args:
        container: A widget container (e.g., VBox, HBox).
        widget_type: The type of widget to search for (e.g., widgets.Button).
        description: The description of the widget to search for.

    Returns:
        The widget if found, otherwise None.
    """
    # Check if the container itself is a widget of the desired type
    if isinstance(container, widget_type) and getattr(container, 'description', '') == description:
        return container

    # If the container has children, search recursively
    if hasattr(container, 'children'):
        for child in container.children:
            found_widget = find_widget_by_description_recursive(child, widget_type, description)
            if found_widget:
                return found_widget

    # If no match is found, return None
    return None

In [13]:
app_ui.box.children

(Box(children=(Button(description='Test Canvas', layout=Layout(height='500px', width='500px'), style=ButtonStyle()),), layout=Layout(height='500px', width='500px')),
 Box(children=(Accordion(children=(Box(children=(Box(children=(Label(value='Probe Width:'), IntSlider(value=334, max=500, min=1)), layout=Layout(align_items='flex-start', flex_flow='column', justify_content='space-around', padding='5px')), Box(children=(Label(value='Probe Height:'), IntSlider(value=50, max=500, min=1)), layout=Layout(align_items='flex-start', flex_flow='column', justify_content='space-around', padding='5px'))), layout=Layout(align_items='flex-start', flex_flow='column', justify_content='space-around')), GridBox(children=(ToggleButton(value=False, description='Audio', icon='volume-up', layout=Layout(height='auto', max_width='100px', width='auto'), tooltip='Enable/disable audio processing'), Box(children=(Label(value='Master Volume (dB):'), FloatSlider(value=0.0, layout=Layout(height='auto', width='100%'), m

In [14]:
# create an IntSlider with a custom property
slider = IntSlider(value=50, min=0, max=100, step=1, description="Test Slider", custom_prop="Test")

In [15]:
vbox = VBox([slider])

In [16]:
vbox.children[0].custom_prop

AttributeError: 'IntSlider' object has no attribute 'custom_prop'

In [17]:
import ipywidgets as widgets

# Function to create a button with a custom tag
def create_button_with_tag(tag):
    button = widgets.Button(description=f"Button {tag}")
    button.tag = tag  # Add a custom property
    return button

# Example usage: create buttons with tags
button1 = create_button_with_tag(tag='button-1')
button2 = create_button_with_tag(tag='button-2')
nested_box = widgets.VBox([button2])
main_container = widgets.VBox([button1, nested_box])

# Function to search recursively by tag
def find_widget_by_tag_recursive(container, tag):
    """
    Recursively search through a container for a widget with a specific custom tag.

    Args:
        container: A widget container (e.g., VBox, HBox).
        tag: The custom tag to search for.

    Returns:
        The widget if found, otherwise None.
    """
    # Check if the container itself has the tag
    if hasattr(container, 'tag') and container.tag == tag:
        return container

    # If the container has children, search recursively
    if hasattr(container, 'children'):
        for child in container.children:
            found_widget = find_widget_by_tag_recursive(child, tag)
            if found_widget:
                return found_widget

    # If no match is found, return None
    return None

# Search for the button with the tag 'button-2'
result = find_widget_by_tag_recursive(main_container, 'button-2')
if result:
    print(f'Found widget with tag: {result.tag}')
else:
    print('Widget not found')


Found widget with tag: button-2
